In [156]:
import pandas as pd
import numpy as np
import regex as re
from datetime import datetime
from time import time
from math import ceil
from datetime import timedelta
import unicodedata  

In [157]:
start_time = time()
#documentating information about the excel files and sheets inside of it.

files = os.listdir()

xslx_files = [x for x in files if x.split('.')[-1]=='xlsx' and x.split('.')[0]!='BI_Financeiro']

def get_sheets_names(x):
    xlsx = pd.ExcelFile(x)
    return xlsx.sheet_names  # see all sheet names

sheets_names = [get_sheets_names(excel) for excel in xslx_files ]

lst_src = list(zip(xslx_files, sheets_names))

for i,v in enumerate(lst_src): 
    exec(f"xlsx_{i} = '{v[0]}'")

df_xlsx = pd.DataFrame(lst_src, columns=['xlsx_file','sheets'])

end_time = time(); print(f'Time: This code took {ceil(end_time-start_time)} seconds to run.'); total_time = end_time-start_time

print('df_xlsx: '); df_xlsx

Time: This code took 1 seconds to run.
df_xlsx: 


,xlsx_file,sheets
0,Controle de Empenhos e NC 2022.xlsx,"[ControleGeral, SintéticoGeral, SintéticoNC, A..."
1,Controle de NC 2022.xlsx,"[SintéticoNC, AnalíticoNC, ÍndiceCorreção]"
2,Emendas2021_resumo.xlsx,[Planilha1]
3,Proponentes.xlsx,[Proponentes]


In [158]:
start_time = time()

df_nm =[
('df_an', 'AnalíticoNC'),
('df_sn', 'SintéticoNC'), 
('df_ic', 'ÍndiceCorreção'),
('df_pr', 'Proponentes'),
('df_cg', 'ControleGeral'),
('df_em', 'Emendas2021_resumo')]

excel_file_1 = 'Controle de NC 2022.xlsx'
excel_file_2 = 'Proponentes.xlsx'
excel_file_3 = 'Controle de Empenhos e NC 2022.xlsx'
excel_file_4 = 'Emendas2021_resumo.xlsx'

df_sheets = pd.DataFrame(df_nm, columns = ['df_name', 'sheet_name'])

df_nm = [y for x,y in df_nm]

df_an = pd.read_excel(excel_file_1,sheet_name=df_nm[0],
usecols="B:J", skiprows=1,parse_dates=True)

df_sn = pd.read_excel(excel_file_1, sheet_name=df_nm[1],
usecols="B:J", skiprows=3,parse_dates=True)

df_ic = pd.read_excel(excel_file_1, sheet_name=df_nm[2],
skiprows=1,usecols="B:E")

df_pr = pd.read_excel(excel_file_2)

df_cg = pd.read_excel(excel_file_3, sheet_name=df_nm[4]
, skiprows=12,usecols="c:w")

df_em= pd.read_excel(excel_file_4)

##----------------------------------------------------------
df_sheets['project'] = 'financeiro'
df_sheets.project.loc[df_sheets['sheet_name']=='Proponentes'] = 'all'

end_time = time(); print(f'Time: This code took {ceil(end_time-start_time)} seconds to run.'); total_time = end_time-start_time
df_sheets

/home/roberto/.local/lib/python3.10/site-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Conditional Formatting extension is not supported and will be removed
  warn(msg)
/home/roberto/.local/lib/python3.10/site-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Slicer List extension is not supported and will be removed
  warn(msg)


Time: This code took 1 seconds to run.


,df_name,sheet_name,project
0,df_an,AnalíticoNC,financeiro
1,df_sn,SintéticoNC,financeiro
2,df_ic,ÍndiceCorreção,financeiro
3,df_pr,Proponentes,all
4,df_cg,ControleGeral,financeiro
5,df_em,Emendas2021_resumo,financeiro


In [159]:
start_time = time()
# criando função 'regexing' p/ subst. nomes abreviados de forma a normalizar as strings


lst_re = [('PRF ','POLICIA RODOVIARIA FEDERAL '),
('DEP. FEDERAL ', ''),
('MCOM ', 'MINISTERIO DAS COMUNICACOES '),
('DEP. FEDERAL', ''),
('PRF ','POLICIA RODOVIARIA FEDERAL '),
('MIN. ','MINISTERIO '),
('SENADORA ',''),
('SENADOR ',''),
('DEP. ','')]
 

def regexing(s):
   for string in lst_re:
        return re.sub(string[0],string[1],s)

lst_proponente = []

for index, prop in enumerate(df_pr['prop_pk1']):
    if df_pr['orgao'][index] == "senado":
        lst_proponente.append('SEN. ' + prop)
    elif  df_pr['orgao'][index] == "camara":
        lst_proponente.append('DEP. ' + prop)
    else: 
        lst_proponente.append(prop)

df_pr['prop_pk2'] = lst_proponente

end_time = time(); print(f'Time: This code took {ceil(end_time-start_time)} seconds to run.'); total_time += end_time-start_time
#Time: This ckode took 1 seconds to run.

Time: This code took 1 seconds to run.


In [160]:
start_time = time()

def strip_accents(cln):
   cln = str(cln)
   return ''.join(c for c in unicodedata.normalize('NFD', cln)if unicodedata.category(c) != 'Mn')

def clean_column_name(cln):
    return strip_accents(cln.strip('[]()').lower().replace(' ','_'))

def rename_df(df):
    new_cln = [clean_column_name(cln) for cln in df.columns.values]
    old_cln = [cln for cln in df.columns.values]
    dct = dict(zip(old_cln, new_cln))
    df.rename(columns=dct,inplace=True)
    lst_df_clns.append(new_cln)
    return df

#creating a list with all dfs from the data dictionary
dfs = [ x for x in df_sheets.df_name]
cmd = str(dfs).replace("'","") 
exec('dfs = {}'.format(cmd))

lst_df_clns = []
for df in dfs:
    rename_df(df)


df_sheets['columns'] = lst_df_clns 

end_time = time(); print(f'Time: This code took {ceil(end_time-start_time)} seconds to run.'); total_time = end_time-start_time

df_sheets

Time: This code took 1 seconds to run.


,df_name,sheet_name,project,columns
0,df_an,AnalíticoNC,financeiro,"[proponente, nota_de_credito, programa, mes, v..."
1,df_sn,SintéticoNC,financeiro,"[nc, proponente, programa, empenho, valor_inic..."
2,df_ic,ÍndiceCorreção,financeiro,"[mes, indice_de_correcao, taxa_selic, observacao]"
3,df_pr,Proponentes,all,"[prop_pk1, uf, partido, orgao, prop_1, prop_2,..."
4,df_cg,ControleGeral,financeiro,"[processo, proponente, contrato, nota_de_empen..."
5,df_em,Emendas2021_resumo,financeiro,"[processo, emenda, valor, pontos_livre_(18_mes..."


In [161]:
start_time = time()


df_em['proponente'] = df_em['emenda']
#fazendo o loop sobre a lista acima para aplicar as funções que normalizam a coluna com as chaves dos proponentes
lst_df = [df_sn, df_an, df_cg, df_em]
for df in lst_df: 
    df['proponente'] = df['proponente'].apply(lambda x:strip_accents(x).upper()).apply(lambda x:regexing(x))
    df['proponente'] = df['proponente'].apply(lambda x:str(x).strip())
#
lst_pr = (df_pr['prop_pk1'], df_pr['prop_1'],df_pr['prop_2'], df_pr['prop_3'],df_pr['prop_4'])
#df_sn['proponente'] = df_sn['proponente'].apply(lambda x:strip_accents(x).upper()).apply(lambda x:regexing(x))
#df_sn['proponente'] = df_sn['proponente'].apply(lambda x:str(x).strip())
for cln in lst_pr:
    cln = cln.apply(lambda x:strip_accents(x).upper()).apply(lambda x:regexing(x))
    
end_time = time(); print(f'Time: This code took {ceil(end_time-start_time)} seconds to run.'); total_time += end_time-start_time

Time: This code took 1 seconds to run.


In [162]:
#cleaning df_em

lamb1 = lambda x: str(x).split('-')[0]
lamb2 = lambda x: str(x).split('(')[0]
lamb3 = lambda x: str(x).replace('DEP. ','')
lamb4 = lambda x: str(x).replace('SEN. ','')
lamb5 = lambda x: str(x).strip()

df_em.proponente = df_em.proponente.apply(lamb1)
df_em.proponente = df_em.proponente.apply(lamb2)
df_em.proponente = df_em.proponente.apply(lamb3)
df_em.proponente = df_em.proponente.apply(lamb4)

#for x in df_em.proponente:
#    for y in lst_re:
#        x = x.replace(y[0],y[1])
#    lst.append(x)



O objetivo inicial do projeto foi limpar os dados relacionados ao os proponentes com a criação de uma coluna nova referente ao nome civil do proponente para que esses valores pudessem ser usados como chave entre diferentes tabelas que serao parametrizadas entre si.

In [163]:
start_time = time()

#criando lista dos DataFrames pelo qual o loop passara referente aos proponentes
dfs = (df_sn, df_an,df_cg, df_em)

#criando as colunas das primary_keys nos 3 dataframes
for df in dfs:
    df['prop_pk1'] = ''
    df['prop_pk2'] = ''
    
#criando lista das colunas referente ao nome dos proponentes 
def filter_key(x):
    return (x == lst_pr[0])|(x==lst_pr[1])|(x==lst_pr[2])|(x==lst_pr[3])

for df in lst_df:
    lst_prop_pk1 = []
    lst_prop_pk2= []
    for index,value in enumerate(df.proponente):
        filt_finding_key =  filter_key(df.proponente.iloc[index])
        result1 = str(df_pr.loc[filt_finding_key].prop_pk1.values).strip("[']")
        result2 = str(df_pr.loc[filt_finding_key].prop_pk2.values).strip("[']")
        lst_prop_pk1.append(result1)
        lst_prop_pk2.append(result2)
    df['prop_pk1'] = lst_prop_pk1
    df['prop_pk2'] = lst_prop_pk2

end_time = time(); print(f'Time: This code took {ceil(end_time-start_time)} seconds to run.'); total_time += end_time-start_time

Time: This code took 2 seconds to run.


In [164]:
start_time = time()
#cleaning analitic sintetic rows

lst = ['SUBTOTAL','TOTAL','NAN']

for value in lst:
    df_sn = df_sn.loc[df_sn['proponente'] != value ]  
    df_em = df_em.loc[df_em['proponente'] != value ]  
    
df_sn.fillna(0,inplace=True)
df_an.fillna(0,inplace=True)

end_time = time(); print(f'Time: This code took {ceil(end_time-start_time)} seconds to run.'); total_time += end_time-start_time

Time: This code took 1 seconds to run.


In [165]:
start_time = time()
dfs = [df_an, df_ic]

for index,df in enumerate(dfs): 
    meses = [mes for mes in df.mes.unique()]
    n_mes = ['01/2021','02/2021','03/2021','04/2021','05/2021','06/2021','07/2021','08/2021','09/2021','10/2021','11/2021','12/2021']
    meses = list(zip(meses, n_mes))

    for index,value in enumerate(meses):
        df['mes'].replace(meses[index][0],meses[index][1],inplace=True)


    df['mes']= pd.to_datetime(df['mes'])
    if (df.mes.dtype == '<M8[ns]') == True:
        print(f'The loop is ok!')
        
    else: print('The loops is broke!')
    
end_time = time(); print(f'Time: This code took {ceil(end_time-start_time)} seconds to run.'); total_time += end_time-start_time

The loop is ok!
The loop is ok!
Time: This code took 1 seconds to run.


In [166]:
#checando se tem dfs sem prop_pk 
start_time = time()

dfs  = [df_an, df_sn,df_em]
lst_no_cv_name = []
for df in dfs:
    for index, value in enumerate(df.loc[df.prop_pk1==''].proponente.values):
        if value not in lst_no_cv_name:
            lst_no_cv_name.append(df.loc[df.prop_pk1==''].proponente.values[index])
        else:pass

print('DF COM OS PROPONENTES QUE NÃO APRESENTARAM CORRESPONDÊNCIA COM A COLUNA DO NOME CIVIL:')
df_pr_nokey = pd.DataFrame()
df_pr_nokey['prop_nokey']= lst_no_cv_name
df_pr_nokey['prop_pk1']= ''
df_pr_nokey['orgao']=''
df_pr_nokey['uf']=''
df_pr_nokey['partido']=''

save = True
if save == True:
    %mkdir trash
    df_pr_nokey.to_excel('trash/props_with_no_key.xlsx')
else:pass

end_time = time(); print(f'Time: This code took {ceil(end_time-start_time)} seconds to run.'); total_time += end_time-start_time

#df_em.loc[df_em.prop_pk1!='']['prop_pk1']

DF COM OS PROPONENTES QUE NÃO APRESENTARAM CORRESPONDÊNCIA COM A COLUNA DO NOME CIVIL:
mkdir: cannot create directory ‘trash’: File exists
Time: This code took 1 seconds to run.


In [167]:
df_sheets

,df_name,sheet_name,project,columns
0,df_an,AnalíticoNC,financeiro,"[proponente, nota_de_credito, programa, mes, v..."
1,df_sn,SintéticoNC,financeiro,"[nc, proponente, programa, empenho, valor_inic..."
2,df_ic,ÍndiceCorreção,financeiro,"[mes, indice_de_correcao, taxa_selic, observacao]"
3,df_pr,Proponentes,all,"[prop_pk1, uf, partido, orgao, prop_1, prop_2,..."
4,df_cg,ControleGeral,financeiro,"[processo, proponente, contrato, nota_de_empen..."
5,df_em,Emendas2021_resumo,financeiro,"[processo, emenda, valor, pontos_livre_(18_mes..."


In [168]:
start_time = time()

#installing the python library to save the data an excel with all the sheets
%rm -r PowerBI_Dataset
%mkdir PowerBI_Dataset
%mkdir PowerBI_Dataset/csv/
%mkdir PowerBI_Dataset/xlsx

path = 'PowerBI_Dataset' 
#putting all dfs inside the excel
for i, v in enumerate(df_sheets['df_name']):
  command1 = (f"{df_sheets['df_name'][i]}.to_csv('{path}/csv/{v}.csv')")
  command2 = (f"{df_sheets['df_name'][i]}.to_excel('{path}/xlsx/{v}.xlsx')")
  exec(command1)
  exec(command2)
  
end_time = time(); print(f'Time: This code took {ceil(end_time-start_time)} seconds to run.'); total_time += end_time-start_time

Time: This code took 1 seconds to run.


In [169]:
%rm -r Data_Dictionary
%mkdir Data_Dictionary
%mkdir Data_Dictionary/csv/
%mkdir Data_Dictionary/xlsx/
%mkdir Data_Dictionary/csv/Columns/
%mkdir Data_Dictionary/xlsx/Columns/

path = 'Data_Dictionary'

for df in ('df_sheets', 'df_xlsx'):
    exec(f"{df}.to_csv('{path}/csv/{df}.csv')")
    exec(f"{df}.to_excel('{path}/xlsx/{df}.xlsx')")


#df_nm = 'df_xlsx'
#df = exec(f'{df_nm}') 
#exec(f"{df}.to_csv('{path}/csv/{df_nm}.csv')")
#exec(f"{df]}.to_excel('{path}/xlsx/{df_nm}.xlsx')")

path2 = 'Columns'

for i, df  in enumerate(df_sheets['df_name']):
       exec(f'df = {df}')
       lst_cl = []
       for cl in df.columns: lst_cl.append(cl)
       dct = {'columns':lst_cl}
       exec(f"{df_sheets['df_name'][i]}_columns = pd.DataFrame(dct)")
       exec(f"{df_sheets['df_name'][i]}_columns['descricao'] = '' ")
       exec(f"{df_sheets['df_name'][i]}_columns.to_csv('{path}/csv/{path2}/{df_sheets['sheet_name'][i]}.csv')")
       exec(f"{df_sheets['df_name'][i]}_columns.to_excel('{path}/xlsx/{path2}/{df_sheets['sheet_name'][i]}.xlsx')")



df_sheets


,df_name,sheet_name,project,columns
0,df_an,AnalíticoNC,financeiro,"[proponente, nota_de_credito, programa, mes, v..."
1,df_sn,SintéticoNC,financeiro,"[nc, proponente, programa, empenho, valor_inic..."
2,df_ic,ÍndiceCorreção,financeiro,"[mes, indice_de_correcao, taxa_selic, observacao]"
3,df_pr,Proponentes,all,"[prop_pk1, uf, partido, orgao, prop_1, prop_2,..."
4,df_cg,ControleGeral,financeiro,"[processo, proponente, contrato, nota_de_empen..."
5,df_em,Emendas2021_resumo,financeiro,"[processo, emenda, valor, pontos_livre_(18_mes..."


In [171]:
final_time= str(timedelta(seconds= total_time)); print(f'Time: The total time to run all the notebook was {final_time}.')
#Time: The total time to run all the notebook is 00:39 minutes.

Time: The total time to run all the notebook was 0:00:02.513593.
